In [1]:
from CGS import solve_cgs;
from CGN import solve_cgn;
from scipy.sparse.linalg import qmr;
import matrixgenerators
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# to be ran if stuff doesnt update with algorithsm
import importlib
import CGS
import CGN
import matrixgenerators

importlib.reload(CGS)
importlib.reload(CGN)
importlib.reload(matrixgenerators)

from CGS import solve_cgs
from CGN import solve_cgn


In [ ]:
First experiment: Calculating how residual error + time taken to run change with matrix size with a fixed condition number.